In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
train_data=pd.read_csv("train.csv")

In [6]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
train=np.array(train_data).T
Xtrain=train[1:]
Ytrain=train[0]
m=Ytrain.size
pixel=len(Xtrain)
print(Xtrain.shape)
Ytrainnew=np.zeros((10,m))
Ytrainnew[Ytrain,np.arange(0,m)]=1
print(Ytrainnew)

(784, 42000)
[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [8]:
def init_params(n_l):
    W1=np.random.randn(n_l,pixel)*0.01
    b1=np.zeros((n_l,1))
    W2=np.random.randn(10,n_l)*0.01
    b2=np.zeros((10,1))
    return W1, b1, W2, b2


In [9]:
def relu(x):
    return np.maximum(0,x)
def softmax(z):
    #print (np.exp(z[:,0]))
    #print("SUM WERE DIVIDING BY axis = 0:",np.sum(np.exp(z[:,0]),axis=0,keepdims=True))
    #print("SUM WERE DIVIDING BY axis = 1:",np.sum(np.exp(z[:,0]),axis=1,keepdims=True))
    #print("Total sum of probabitlities:",np.sum(np.exp(z)/np.sum(np.exp(z),axis=0,keepdims=True),axis=0,keepdims=True))
    return (np.exp(z)/np.sum(np.exp(z),axis=0,keepdims=True))
def del_relu(x):
    return np.where(x>0,1,0)

In [10]:
def forward_prop(W1,b1,W2,b2,X):
    Z1=W1.dot(X)+b1
    A1=relu(Z1)
    Z2=W2.dot(A1)+b2
    A2=softmax(Z2)
    #print("A2 SUm",np.sum(A2,axis=0,keepdims=True))
    cache = {"Z1":Z1,"A1":A1,"Z2":Z2,"A2":A2}
    return cache, A2


In [11]:
def back_prop(X,Y,A2,W1,b1,W2,b2,cache):
    dZ2=A2-Y
    dW2=(1/m)*dZ2.dot(cache["A1"].T)
    db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1=W2.T.dot(dZ2)*del_relu(cache["Z1"])
    dW1=(1/m)*dZ1@X.T
    db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
    return dW1,db1,dW2,db2

In [12]:
def compute_cost(Y,A):
    return (1/m)*np.sum(Y*np.log(A)+(1-Y)*np.log(1-A))
def update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,learning_rate):
    W1-=learning_rate*dW1
    b1-=learning_rate*db1
    W2-=learning_rate*dW2
    b2-=learning_rate*db2
    return W1,b1,W2,b2

In [13]:
def learn(X,Y,num_its,learning_rate):
    W1, b1, W2, b2=init_params(20)
    for i in range(num_its):
        cache,A2 = forward_prop(W1,b1,W2,b2,X)
        cost=compute_cost(Y,A2)
        dW1,db1,dW2,db2=back_prop(X,Y,A2,W1,b1,W2,b2,cache)
        W1,b1,W2,b2=update_params(W1,b1,W2,b2,dW1,db1,dW2,db2,learning_rate)
        if i%10==0:print("Cost after",i,"th iteration is",cost)
        #print(A2[:,0])
        
    return W1,b1,W2,b2

In [14]:
W1,b1,W2,b2=learn(Xtrain,Ytrainnew,1000,0.001)

Cost after 0 th iteration is -3.4637825569756915
Cost after 10 th iteration is -2.298978676417156
Cost after 20 th iteration is -1.5755886268403063
Cost after 30 th iteration is -1.2381242449126932
Cost after 40 th iteration is -1.0650424344345688
Cost after 50 th iteration is -0.9592828741669766
Cost after 60 th iteration is -0.8869013087403202
Cost after 70 th iteration is -0.8335189106408297
Cost after 80 th iteration is -0.7920114713946993
Cost after 90 th iteration is -0.7586850889990271
Cost after 100 th iteration is -0.7318899883723646
Cost after 110 th iteration is -0.7127670249387111
Cost after 120 th iteration is -0.7032348467293547
Cost after 130 th iteration is -0.6865426873052768
Cost after 140 th iteration is -0.660283779027775
Cost after 150 th iteration is -0.6417729537868397
Cost after 160 th iteration is -0.6281144393639474
Cost after 170 th iteration is -0.616478816820924
Cost after 180 th iteration is -0.6060192953708443
Cost after 190 th iteration is -0.59644206716

In [15]:
Ytrain[0]

np.int64(1)

In [16]:
cache, A2=forward_prop(W1,b1,W2,b2,Xtrain[:,3].reshape(784,1))

In [17]:
print(A2)

[[1.46894463e-01]
 [7.96712968e-05]
 [1.29025044e-01]
 [1.09523837e-02]
 [3.51909966e-01]
 [1.75599120e-01]
 [7.93561536e-02]
 [9.35805676e-03]
 [1.54086814e-02]
 [8.14164605e-02]]
